In [1]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns
import statistics 
from collections import OrderedDict
import csv
import re
from collections import OrderedDict

In [2]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
infiles = filedialog.askopenfilename(multiple=True, title='load posthoc-predictions.npy')

%gui tk

infiles

('K:/KayCei/HDC/AH4-KO/022024-Lights Off/predict_video_exp_16.npy',)

In [3]:
filepath = infiles[0]
parts = filepath.split('/')
print(parts)
parts1= re.sub(r'\d', '', parts[-2])
print(parts1)
directory_path = '/'.join(parts[:-1])  # Join all parts except the last two
file_name = ''.join([parts[-3],parts1])  # Join the last two parts
print("Directory Path:", directory_path)
print("File Name:", file_name)


['K:', 'KayCei', 'HDC', 'AH4-KO', '022024-Lights Off', 'predict_video_exp_16.npy']
-Lights Off
Directory Path: K:/KayCei/HDC/AH4-KO/022024-Lights Off
File Name: AH4-KO-Lights Off


In [4]:
Random_List = [] #creates a array known as random list 
for f in range(len(infiles)):
    arr = np.load(infiles[f])
    Random_List.extend(arr) 
#print(Random_List)

In [5]:
duration_in_deciseconds = len(Random_List)
duration_in_minutes = duration_in_deciseconds / 600
duration_in_frames = duration_in_deciseconds * 6

print('This video is', duration_in_deciseconds, 'deciseconds, which is', duration_in_minutes, 'minutes, which is', duration_in_frames, 'frames')
#print(Random_List)

This video is 39215 deciseconds, which is 65.35833333333333 minutes, which is 235290 frames


In [6]:
# Define behavior labels with numbers starting from 0.0
behavior_labels = OrderedDict({
    'torso_groom': 0, 'investigate_0': 1, 'unsure_0': 2, 'wall_rear_0': 3, 'rear_0': 4,
    'unsure_2': 5, 'rear': 6, 'investigate_1': 7, 'investigate_2': 8, 'right_itch': 9,
    'investigate_3': 10, 'sleep_scrunch_0': 11, 'sleep_scrunch_1': 12, 'wall_rear_1': 13, 
    'right_body_groom': 14,'face_groom_1': 15, 'dive_scrunch': 16, 'head_groom': 17, 'left_orient': 18, 'left_investigate': 19,
    'face_groom_2': 20, 'left_body groom': 21, 'left_itch_1': 22, 'left_itch_2': 23, 
    'face_groom_3': 24,'paw_groom': 25, 'locomotion': 26, 'right_forepaw_orient': 27, 'right_orient': 28
})

In [7]:
from collections import Counter

# Your list to store behavior labels and their corresponding indices
BehaviorTime = []

# Mapping values to behavior labels
for index, value in enumerate(Random_List):
    behavior_label = next(key for key, val in behavior_labels.items() if val == value)
    BehaviorTime.append((behavior_label, index))

# Count occurrences of each behavior
behavior_counts = Counter(behavior for behavior, _ in BehaviorTime)

# Print behavior counts
for behavior, count in behavior_counts.items():
    print(f"{behavior}: {count}")


locomotion: 12233
left_orient: 2418
investigate_2: 15602
investigate_3: 1121
dive_scrunch: 467
wall_rear_0: 276
right_orient: 2110
investigate_1: 1471
face_groom_3: 48
right_forepaw_orient: 625
unsure_2: 508
right_itch: 573
face_groom_1: 519
unsure_0: 9
left_body groom: 166
head_groom: 115
left_itch_1: 89
paw_groom: 87
right_body_groom: 94
left_itch_2: 237
wall_rear_1: 19
rear: 261
face_groom_2: 95
sleep_scrunch_1: 5
investigate_0: 14
rear_0: 31
left_investigate: 15
torso_groom: 5
sleep_scrunch_0: 2


In [8]:
BehaviorTime

[('locomotion', 0),
 ('locomotion', 1),
 ('locomotion', 2),
 ('locomotion', 3),
 ('locomotion', 4),
 ('locomotion', 5),
 ('left_orient', 6),
 ('locomotion', 7),
 ('investigate_2', 8),
 ('investigate_2', 9),
 ('investigate_2', 10),
 ('locomotion', 11),
 ('locomotion', 12),
 ('locomotion', 13),
 ('locomotion', 14),
 ('locomotion', 15),
 ('locomotion', 16),
 ('locomotion', 17),
 ('left_orient', 18),
 ('left_orient', 19),
 ('left_orient', 20),
 ('investigate_3', 21),
 ('dive_scrunch', 22),
 ('left_orient', 23),
 ('locomotion', 24),
 ('dive_scrunch', 25),
 ('wall_rear_0', 26),
 ('right_orient', 27),
 ('locomotion', 28),
 ('locomotion', 29),
 ('locomotion', 30),
 ('locomotion', 31),
 ('locomotion', 32),
 ('locomotion', 33),
 ('locomotion', 34),
 ('locomotion', 35),
 ('locomotion', 36),
 ('locomotion', 37),
 ('locomotion', 38),
 ('locomotion', 39),
 ('locomotion', 40),
 ('locomotion', 41),
 ('locomotion', 42),
 ('locomotion', 43),
 ('locomotion', 44),
 ('locomotion', 45),
 ('locomotion', 46),

In [9]:
# Extract unique behaviors
behaviors = [behavior for behavior in behavior_labels.keys()]

# Create a DataFrame with time as the index
df = pd.DataFrame(index=range(max(time for _, time in BehaviorTime) + 1))

# Iterate over behaviors to create columns and mark 1 or 0
for behavior in behaviors:
    behavior_column = [1 if b == behavior else 0 for b, _ in BehaviorTime]
    df[behavior] = behavior_column

# Set the time column and divide each time by 10
df['time'] = [time/10 for _, time in BehaviorTime]

# Reorder columns
df = df[['time'] + behaviors]

# Specify the path to save the CSV file
file_path = 'K:/KayCei/behavior_data.csv'

# Save to CSV
df.to_csv(file_path, index=False)

print(f"CSV file saved to: {file_path}")

CSV file saved to: K:/KayCei/behavior_data.csv


In [10]:
df

time  torso_groom  investigate_0  unsure_0  wall_rear_0  rear_0  \
0         0.0            0              0         0            0       0   
1         0.1            0              0         0            0       0   
2         0.2            0              0         0            0       0   
3         0.3            0              0         0            0       0   
4         0.4            0              0         0            0       0   
...       ...          ...            ...       ...          ...     ...   
39210  3921.0            0              0         0            0       0   
39211  3921.1            0              0         0            0       0   
39212  3921.2            0              0         0            0       0   
39213  3921.3            0              0         0            0       0   
39214  3921.4            0              0         0            0       0   

       unsure_2  rear  investigate_1  investigate_2  ...  left_investigate  \
0             0     0              0              0  ...                 0   
1             0     0              0              0  ...                 0   
2             0     0              0              0  ...                 0   
3             0     0              0              0  ...                 0   
4             0     0              0              0  ...                 0   
...         ...   ...            ...            ...  ...               ...   
39210         0     0              0              1  ...                 0   
39211         0     0              0              1  ...                 0   
39212         0     0              0              1  ...                 0   
39213         0     0              0              1  ...                 0   
39214         0     0              0              1  ...                 0   

       face_groom_2  left_body groom  left_itch_1  left_itch_2  face_groom_3  \
0                 0                0            0            0             0   
1                 0                0            0            0             0   
2                 0                0            0            0             0   
3                 0                0            0            0             0   
4                 0                0            0            0             0   
...             ...              ...          ...          ...           ...   
39210             0                0            0            0             0   
39211             0                0            0            0             0   
39212             0                0            0            0             0   
39213             0                0            0            0             0   
39214             0                0            0            0             0   

       paw_groom  locomotion  right_forepaw_orient  right_orient  
0              0           1                     0             0  
1              0           1                     0             0  
2              0           1                     0             0  
3              0           1                     0             0  
4              0           1                     0             0  
...          ...         ...                   ...           ...  
39210          0           0                     0             0  
39211          0           0                     0             0  
39212          0           0                     0             0  
39213          0           0                     0             0  
39214          0           0                     0             0  

[39215 rows x 30 columns]